# 1. Cargamos las librerías

In [3]:
from pprint import pprint
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.axes as ax
import seaborn as sns
import plotly.express as px
import scipy
from scipy import stats
import os

# 2. Cargamos los datos extraídos por ciudades entre 2021 y 2025:

## - Pontevedra 

In [4]:
df_ponte = pd.concat(objs = [pd.read_csv(x) for x in os.listdir() if x.startswith("ponte") and x.endswith(".csv")],
                     ignore_index = True)

df_ponte = df_ponte.drop(labels = "Unnamed: 0", axis = 1)

df_ponte["Fecha"] = df_ponte["Fecha"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d"))

df_ponte["Year"] = df_ponte["Fecha"].apply(lambda x : x.year)

def estacion(m):
    if   m in [12, 1, 2]: return "Invierno"
    elif m in [3, 4, 5]:  return "Primavera"
    elif m in [6, 7, 8]:  return "Verano"
    else:                 return "Otoño"

df_ponte["Estacion"] = df_ponte["Fecha"].dt.month.apply(estacion)
df_ponte["Mes"] = df_ponte["Fecha"].apply(lambda x : x.month)
rename_columns = {"Momento (UNIX) de la vel. máx. del veinto" : "Momento (UNIX) de la vel. máx. del viento",
                  "Profundidad de la neive" : "Profundidad de la nieve",
                  "Vel. ráfaga de viento max." : "Vel. ráfaga de viento máx."}

df_ponte = df_ponte.rename(columns = rename_columns)

df_ponte

ValueError: No objects to concatenate